In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image


/home/setup/miniconda3/envs/vision/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


model_name="Qwen/Qwen2-VL-2B-Instruct"

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
   model_name , torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained(
  model_name 
)


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.41it/s]


In [ ]:

#推論関数
def process_image(
    model,processor,
    img_path,
query="中身を解析して",
max_new_tokens=128,
):

    # 画像の準備
    image = Image.open(img_path)

    # メッセージの準備
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                },
                {"type": "text", "text": query},
            ],
        }
    ]

    # 入力の準備
    text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(
        text=[text_prompt], 
        images=[image], 
        padding=True, 
        return_tensors="pt"
    )
    inputs = inputs.to("cuda")

    # 推論の実行
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )
    return output_text

#推論
path="images/particle.png"
query="この画像は何?"
text=process_image(model,processor,path,query)